<a href="https://colab.research.google.com/github/JFrois/Envio_de_casos/blob/main/Envio_de_casos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Instalações necessárias

* Pips install para conseguir realizar os imports



In [ ]:
!pip install pandas gspread openpyxl gspread_dataframe xlsxwriter


# 2. Imports das bibliotecas

* Imports necessários para autenticação, pandas, datetime, google sheets

In [ ]:
import gspread
import pandas as pd
import time
from google.colab import auth, drive
from google.auth import default
import os
from datetime import datetime
import re

# 3. Código para consolidação das planilhas de ITBI



* Período: 2019 a 2025
* Filtra por tipo transação = Compra e venda
* Exclui colunas que não fazem sentido ter:
Referência
Valor de Transação (declarado pelo contribuinte)
Valor Venal de Referência
Proporção Transmitida (%)
Valor Venal de Referência (Proporcional)
Base de Cálculo adotada
Tipo de Financiamento
Valor Financiado
Situação SQL
Área do Terreno (m2)
Testada (m)
Fração Ideal
Área Construída (m2)
Uso (IPTU)
Descrição do uso (IPTU)
Padrão (IPTU)
ACC (IPTU)
ACC (IPTU).1
escrição do padrão (IPTU)
Unnamed:28

In [ ]:
# Lista os Drives disponíveis
shared_drives_path = "informe seu drive"
#print("Drives disponíveis:")
#print(os.listdir(shared_drives_path))


drive.mount('/content/drive', force_remount=True)

# Autenticação para acessar o Google Drive e Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


# Caminho da pasta onde estão os arquivos Excel
data_file_folder = os.path.join('Informe caminho da pasta')


# Define os arquivos e abas correspondentes
file_paths = [
    os.path.join(data_file_folder, 'GUIAS DE ITBI PAGAS (2019).xlsx'),
    os.path.join(data_file_folder, 'GUIAS DE ITBI PAGAS (2020).xlsx'),
    os.path.join(data_file_folder, 'GUIAS DE ITBI PAGAS (2021).xlsx'),
    os.path.join(data_file_folder, 'GUIAS DE ITBI PAGAS (2022).xlsx'),
    os.path.join(data_file_folder, 'GUIAS DE ITBI PAGAS (2023).xlsx'),
    os.path.join(data_file_folder, 'GUIAS DE ITBI PAGAS (2024).xlsx'),
    os.path.join(data_file_folder, 'GUIAS DE ITBI PAGAS (2025).xlsx'),
]

# Definindo as abas por ano
sheet_names = {
    2019: ['JAN-2019', 'FEV-2019', 'MAR-2019', 'ABR-2019', 'MAI-2019', 'JUN-2019', 'JUL-2019',
           'AGO-2019', 'SET-2019', 'OUT-2019', 'NOV-2019', 'DEZ-2019'],
    2020: ['JAN-2020', 'FEV-2020', 'MAR-2020', 'ABR-2020', 'MAI-2020', 'JUN-2020', 'JUL-2020',
           'AGO-2020', 'SET-2020', 'OUT-2020', 'NOV-2020', 'DEZ-2020'],
    2021: ['JAN-2021', 'FEV-2021', 'MAR-2021', 'ABR-2021', 'MAI-2021', 'JUN-2021', 'JUL-2021',
           'AGO-2021', 'SET-2021', 'OUT-2021', 'NOV-2021', 'DEZ-2021'],
    2022: ['JAN-2022', 'FEV-2022', 'MAR-2022', 'ABR-2022', 'MAI-2022', 'JUN-2022', 'JUL-2022',
           'AGO-2022', 'SET-2022', 'OUT-2022', 'NOV-2022', 'DEZ-2022'],
    2023: ['JAN-2023', 'FEV-2023', 'MAR-2023', 'ABR-2023', 'MAI-2023', 'JUN-2023', 'JUL-2023',
           'AGO-2023', 'SET-2023', 'OUT-2023', 'NOV-2023', 'DEZ-2023'],
    2024: ['JAN-2024', 'FEV-2024', 'MAR-2024', 'ABR-2024', 'MAI-2024', 'JUN-2024', 'JUL-2024',
           'AGO-2024', 'SET-2024', 'OUT-2024', 'NOV-2024','DEZ-2024'],
    2025: ['JAN-2025', 'FEV-2025','MAR-2025'],
}

# Variável para armazenar os dados consolidados
history_data = pd.DataFrame()

# Loop para consolidar dados de todas as abas de todas as planilhas
for file_path, year in zip(file_paths, sheet_names.keys()):
    for sheet_name in sheet_names[year]:
        try:
            table = pd.read_excel(file_path, sheet_name=sheet_name)
            table.columns = table.columns.str.strip()

            if 'Natureza de Transação' not in table.columns:
                print(f"Pulando {sheet_name} de {file_path} - Coluna 'Natureza de Transação' não encontrada")
                continue

            history_data = pd.concat([history_data, table], axis=0, ignore_index=True)
            print(f"Carregado: {sheet_name} de {file_path}")
        except Exception as e:
            print(f"Erro ao carregar {sheet_name} de {file_path}: {e}")


# Remoção das colunas desnecessárias
columns_to_remove = [
    'Referência', 'Valor de Transação (declarado pelo contribuinte)', 'Valor Venal de Referência',
    'Proporção Transmitida (%)', 'Valor Venal de Referência (Proporcional)', 'Base de Cálculo adotada',
    'Tipo de Financiamento', 'Valor Financiado', 'Situação SQL', 'Área do Terreno (m2)', 'Testada (m)',
    'Fração Ideal', 'Área Construída (m2)', 'Uso (IPTU)', 'Descrição do uso (IPTU)', 'Padrão (IPTU)',
    'ACC (IPTU)', 'ACC (IPTU).1', 'escrição do padrão (IPTU)', 'Unnamed:28'
    ]

history_data = history_data.drop(columns=columns_to_remove, errors='ignore')

# Filtra os dados pela coluna "Natureza de Transação"
filtered_data = history_data[history_data['Natureza de Transação'] == '1.Compra e venda']

# Padroniza a coluna "Data de Transação"
filtered_data['Data de Transação'] = pd.to_datetime(filtered_data['Data de Transação'], format='%d/%m/%Y', errors='coerce')

# Padroniza a coluna "Cartório de Registro"
filtered_data['Cartório de Registro'] = filtered_data['Cartório de Registro'].str.replace('CARTORIO DE REGISTRO DE IMOVEL', '', regex=False).str.strip()

# Padroniza a coluna "Cartório de Registro"
filtered_data['Cartório de Registro'] = filtered_data['Cartório de Registro'].str.replace('º Cartório de Registro de Imóvel', '', regex=False).str.strip()

# Salva os dados consolidados em um novo arquivo Excel
output_file = os.path.join(data_file_folder, 'ITBI SP CONSOLIDADO(2019 - 2025).xlsx')
filtered_data.to_excel(output_file, index=False)

print(f"Consolidação concluída. Arquivo salvo em: {output_file}")

# 4. Quais casos de SP com número de matricula tivemos movimentação

*   Neste código é realizado a verificação de quais casos tivemos movimentação de ITBI de SP

**Como é feito a verificação:**

* **Informações necessárias:** Data primeira visita e número de matricula


1.   Verifica se o número de matricula do imóvel que consta na nossa base é igual planilha de ITBI
2.   Se a verificação acima for verdadeira, o códio faz a verificação da data de primeira visita e caso a data da transação for igual ou maior que a data da primeira visita consta como motivmentação = Sim

In [ ]:
shared_drives_path = "/content/drive/Shareddrives"

drive.mount('/content/drive', force_remount=True)

# Autenticação para acessar o Google Drive e Google Sheets
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Caminho do arquivo Excel no Google Drive
itbi_excel_path = 'Informe caminho da pasta'

# Carregar os dados do ITBI (Excel)
itbi_data = pd.read_excel(itbi_excel_path)

# Abrir a planilha Google Sheets de CasosSP
casos_sp_sheet = gc.open('Verificação casos para envio')  # Nome da planilha
casos_sp_aba = casos_sp_sheet.worksheet('CasosSP - Com matricula')  # Nome da aba
casos_data = pd.DataFrame(casos_sp_aba.get_all_records())

# Mapeamento de meses por extenso para números
meses_map = {
    "janeiro": "01", "fevereiro": "02", "março": "03", "abril": "04", "maio": "05",
    "junho": "06", "julho": "07", "agosto": "08", "setembro": "09", "outubro": "10",
    "novembro": "11", "dezembro": "12"
}

# Função para substituir meses por extenso por números
def substituir_meses(data_str):
    if not isinstance(data_str, str):
        return data_str
    for mes, num in meses_map.items():
        data_str = re.sub(rf'\b{mes}\b', num, data_str, flags=re.IGNORECASE)
    return data_str

# Atualizar a coluna de datas na planilha CasosSP
if 'primeira_visita' in casos_data.columns:
    casos_data['primeira_visita'] = (
        casos_data['primeira_visita']
        .apply(substituir_meses)
        .apply(lambda x: pd.to_datetime(x, errors='coerce').strftime('%d/%m/%Y') if pd.to_datetime(x, errors='coerce') else None)
    )

# Padronizar datas na planilha ITBI
itbi_data['Data de Transação'] = itbi_data['Data de Transação'].apply(
    lambda x: pd.to_datetime(x, errors='coerce').strftime('%d/%m/%Y') if pd.to_datetime(x, errors='coerce') else None
)

# Adicionar colunas para os resultados, caso não existam
if 'Movimentação' not in casos_data.columns:
    casos_data['Movimentação'] = 'Não'
if 'Data de Movimentação' not in casos_data.columns:
    casos_data['Data de Movimentação'] = None

# Processar os dados
for idx, caso in casos_data.iterrows():
    # Obter informações do caso
    primeira_visita = pd.to_datetime(caso['primeira_visita'], format='%d/%m/%Y', errors='coerce')
    matricula_caso = caso.get('numero_matricula')

    if pd.isna(primeira_visita) or not matricula_caso:
        continue  # Ignorar linhas com dados insuficientes

    # Filtrar as movimentações do ITBI
    movimentacao = itbi_data[
        (pd.to_datetime(itbi_data['Data de Transação'], format='%d/%m/%Y', errors='coerce') >= primeira_visita) &
        (itbi_data['Matrícula do Imóvel'] == matricula_caso)
    ]

    # Se encontrar movimentação, atualize os resultados
    if not movimentacao.empty:
        casos_data.at[idx, 'Movimentação'] = 'Sim'
        casos_data.at[idx, 'Data de Movimentação'] = movimentacao['Data de Transação'].iloc[0]

# Garantir que as colunas de data estão formatadas como string no padrão esperado
casos_data['primeira_visita'] = casos_data['primeira_visita'].fillna('').astype(str)
casos_data['Data de Movimentação'] = casos_data['Data de Movimentação'].fillna('').astype(str)

# Atualizar os valores na planilha Google Sheets
casos_sp_aba.update([casos_data.columns.values.tolist()] + casos_data.values.tolist())

print("Planilha de CasosSP atualizada com sucesso!")

# 5. Envio de casos para API



*   Neste código, é enviado os casos da planilha de estudo para a nossa API da Zendesk



In [ ]:
# Caminho completo do arquivo de controle
#caminho_arquivo = r"C:\Users\juan.frois\Desktop\Python\API planilha estudo\ultimo_caso.txt"

shared_drives_path = "/content/drive/Shareddrives"
drive.mount('/content/drive', force_remount=True)
caminho_arquivo = 'Informe caminho da pasta'

# Autenticação e acesso às planilhas
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# Acesso às planilhas
planilha = gc.open("Estudo Bypass")
aba_backlog = planilha.worksheet("Enviar para API")

planilhaApi = gc.open("Estudo Bypass")
aba_api = planilhaApi.worksheet("Enviado para API")

# Carrega o índice do último caso enviado e a data do envio
def carregar_ultimo_caso():
    if os.path.exists(caminho_arquivo):
        with open(caminho_arquivo, "r") as f:
            linhas = f.readlines()
            ultimo_caso = int(linhas[0].strip())
            data_envio = linhas[1].strip() if len(linhas) > 1 else "Data não registrada"
            return ultimo_caso, data_envio
    return 0, "Data não registrada"  # Se o arquivo não existe, começa do caso 0

# Salva o índice do último caso enviado e a data do envio
def salvar_ultimo_caso(indice):
    data_atual = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(caminho_arquivo, "w") as f:
        f.write(f"{indice}\n{data_atual}")

# Função para copiar dados da "Backlog" para a "API" na mesma ordem, evitando duplicatas completas
def copiar_dados():
    try:
        # Carrega os dados da aba "Backlog" ignorando o cabeçalho
        backlog_lista = aba_backlog.get_all_values()[1:]  # Ignora o cabeçalho
        backlog_df = pd.DataFrame(backlog_lista, columns=aba_backlog.row_values(1))  # DataFrame com cabeçalho

        # Carrega os dados existentes da aba "API" ignorando o cabeçalho
        api_lista = aba_api.get_all_values()[1:]  # Ignora o cabeçalho
        api_df = pd.DataFrame(api_lista, columns=aba_api.row_values(1)) if api_lista else pd.DataFrame()

        # Cria conjuntos com os valores da coluna A para comparação
        backlog_coluna_a = set(backlog_df.iloc[:, 0])  # Coluna A da Backlog
        api_coluna_a = set(api_df.iloc[:, 0]) if not api_df.empty else set()  # Coluna A da API

        # Filtra apenas os casos da Backlog que não estão na API
        backlog_nao_enviados = backlog_df[~backlog_df.iloc[:, 0].isin(api_coluna_a)]

        # Verifica se há novos casos para enviar
        if not backlog_nao_enviados.empty:
            # Seleciona até 90 casos para enviar por vez
            casos_para_enviar = backlog_nao_enviados.head(90).values.tolist()

            # Define a última linha preenchida na aba "API"
            ultima_linha_api = len(api_df) + 2  # +2 por causa do cabeçalho

            # Atualiza a aba "API" com os novos casos
            aba_api.update(f"A{ultima_linha_api}", casos_para_enviar)
            print(f"{len(casos_para_enviar)} novos casos foram copiados para a aba 'API'.")

            # Salva o índice do último caso enviado
            salvar_ultimo_caso(len(backlog_df) - len(backlog_nao_enviados))
        else:
            print("Não há novos casos na aba 'Backlog' para enviar para a aba 'API'.")

    except Exception as e:
        print(f"Ocorreu um erro ao tentar copiar os dados: {e}")

# Executa continuamente com timer de 600 segundos
def executar_com_timer(intervalo=600):  # 600 segundos = 10 minutos
    while True:
        print(f"Iniciando verificação: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        copiar_dados()
        print(f"Próxima verificação em {intervalo / 60} minutos.\n")
        time.sleep(intervalo)

# Inicia o processo com timer de 600 segundos
executar_com_timer(600)
